In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
df_projects = client.to_pandas(f"""
    SELECT *
    FROM projects_v1
    WHERE project_source = 'OP_ATLAS'
""")
df_projects.tail()

,project_id,project_source,project_namespace,project_name,display_name,description
4222,sraRgd9n5/Xx1Ujd6PxC1UH86UNavOe60aNkdMS49iQ=,OP_ATLAS,,0xcf4d4a83b5344aea1e5e92b0922c8e0a5df3e9f16a24...,Blabberix,"Hello, I am glad to have the opportunity to te..."
4223,OazItCgMb7ibuHz+F3InAUt53WyVQrMw/D6bUaJsls4=,OP_ATLAS,,0x09ac44ceabc28694eaa828c6d834c1bfe21681f2d21a...,Cosmic Alpha,A layer3 project which based on @layer3xyz eco...
4224,Ljrto3jua76uWVqE1Z58zyABGJklhzx6aGFbP9rToPM=,OP_ATLAS,,0xf6f729f9b48cf8ea0d8ddab094cb489c9e8e97851d08...,AUDSC StableCoin and Yield Protocol,We are developing an AUD stablecoin project th...
4225,Yeh/6rDqxARpmlrTPMVCFsXUVOTRdBxiaUjb6nZOId4=,OP_ATLAS,,0x695b64fd61d7dc6a335702d21c124e9ecd11e4591041...,google,test
4226,olryGNRpQHPO0vc3xcB6Z6PIeW4LVAXgpyC893du7Y4=,OP_ATLAS,,0xf554623c765e4c706f6cf60e92ab518d87948d052fed...,Any Inu,The most multichain dogcoin. One fixed supply ...


In [6]:
df_metrics = client.to_pandas(f"""

    SELECT
        p.project_name,
        p.display_name,
        tm.sample_date,
        tm.amount,
        MAX(pbc.collection_name = 'op-retrofunding-4') AS in_rf4
    FROM timeseries_metrics_by_project_v0 AS tm
    JOIN metrics_v0 AS m ON tm.metric_id = m.metric_id
    JOIN projects_v1 AS p ON tm.project_id = p.project_id
    JOIN projects_by_collection_v1 AS pbc ON p.project_id = pbc.project_id
    WHERE
        m.metric_name = 'OPTIMISM_transactions_daily'
        AND p.project_source = 'OSS_DIRECTORY'
        AND tm.sample_date >= DATE '2024-01-01'
        AND tm.sample_date < DATE '2024-07-01'
    GROUP BY 1,2,3,4
        
""")

df_metrics.tail()

,project_name,display_name,sample_date,amount,in_rf4
41847,codingshdev,CodingshDev,2024-04-20,2,False
41848,codingshdev,CodingshDev,2024-02-13,2,False
41849,codingshdev,CodingshDev,2024-06-03,1,False
41850,codingshdev,CodingshDev,2024-06-15,5,False
41851,codingshdev,CodingshDev,2024-06-25,2,False


In [8]:
df_metrics[df_metrics['in_rf4'] == False]

,project_name,display_name,sample_date,amount,in_rf4
0,originprotocol,Origin Dollar,2024-04-08,1,False
1,filosofiacodigo,FilosofiaCodigo,2024-02-11,16,False
2,filosofiacodigo,FilosofiaCodigo,2024-02-16,2,False
3,filosofiacodigo,FilosofiaCodigo,2024-03-26,1,False
4,filosofiacodigo,FilosofiaCodigo,2024-05-25,2,False
...,...,...,...,...,...
41847,codingshdev,CodingshDev,2024-04-20,2,False
41848,codingshdev,CodingshDev,2024-02-13,2,False
41849,codingshdev,CodingshDev,2024-06-03,1,False
41850,codingshdev,CodingshDev,2024-06-15,5,False
